In [2]:
import pandas as pd
from pathlib import Path

results_dir = Path.cwd().parent.parent / "data" / "results"

raw = pd.read_csv(results_dir / "logistic_regression_raw_metrics.csv", index_col=0)
processed = pd.read_csv(results_dir / "logistic_regression_processed_metrics.csv", index_col=0)

metrics = ["precision", "recall", "f1-score", "accuracy", "time_seconds"]

# Tạo dict để tránh lỗi transpose
data = {
    "Raw Logistic": [
        raw.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
        else raw.loc[m, raw.columns[0]]
        for m in metrics
    ],
    "Processed Logistic": [
        processed.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
        else processed.loc[m, processed.columns[0]]
        for m in metrics
    ]
}

# Metric là index, không phải cột
comparison = pd.DataFrame(data, index=metrics)

# Chuyển vị
comparison_T = comparison.transpose()

# Lưu file không index
comparison_T.to_csv(results_dir / "logistic_comparison.csv", index=False)

comparison_T.head()


,precision,recall,f1-score,accuracy,time_seconds
Raw Logistic,0.89497,0.946029,0.919791,0.946029,0.336035
Processed Logistic,0.94534,0.685294,0.774441,0.685294,0.059329
